# Notebook to test and illustrate FAST algorithm

In [1]:
%load_ext autoreload
%autoreload 2

### Tests on single image

In [2]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

import fast
import opticalFlow

# opencv example

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture('imgs/person_dog.mp4')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

### Test grid on video

In [4]:
cap = cv2.VideoCapture('imgs/person_dog.mp4')

START_IDX = 60
END_IDX = 70
IDX = 0

ret, frame = cap.read()
result_gif = 'imgs/person_dog_kp_detector_grid_plus_adaptive_thresholds_2.gif'

imgs = []
skip = 3
nb_keypoints = 1000
cols = 4
rows = 3

while ret:
    if IDX % skip != 0:
        ret, frame = cap.read()
        IDX += 1
        continue
    if START_IDX <= IDX < END_IDX:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        keypoints = fast.detect_with_adaptive_threshold_and_grid(
            gray, nb_keypoints, N=12, step=5, epsilon=nb_keypoints*0.1,
            percentage=0.10, init_thresholds=None, rows=rows, cols=cols)
        img_with_kp = fast.draw(frame, keypoints)
        imgs.append(Image.fromarray(cv2.cvtColor(img_with_kp, cv2.COLOR_BGR2RGB)))
    elif IDX >= END_IDX:
        break
    ret, frame = cap.read()
    IDX += 1
    print(f'IDX={IDX}', end='\r')

imgs[0].save(result_gif, format='GIF',
             append_images=imgs[1:],
             save_all=True, loop=0)

cap.release()
cv2.destroyAllWindows()